上一节提出的三个问题，可以通过仅调用一次Pregel算子就可以解决，下面先利用Pregel计算一次上节的最远距离，然后再说明各参数的含义

In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.graphx._
val sparkConf: SparkConf = new SparkConf().setMaster("local[2]")
val sc: SparkContext = new SparkContext(sparkConf)

sparkConf = org.apache.spark.SparkConf@78edcfc0
sc = org.apache.spark.SparkContext@7e3c0318


org.apache.spark.SparkContext@7e3c0318

In [2]:
//为了方便，这里各节点属性就直接设置为0了
val verticesRDD=sc.parallelize(Array((0L,0),(1L,0),(2L,0),(3L,0),(4L,0),(5L,0)))
val edgesRDD=sc.parallelize(Array(Edge(0L,1L,"r1"),Edge(0L,2L,"r1"),Edge(1L,3L,"r2"),Edge(2L,3L,"r3"),Edge(2L,4L,"r2"),Edge(5L,4L,"r3")))
val graph=Graph(verticesRDD,edgesRDD)

verticesRDD = ParallelCollectionRDD[0] at parallelize at <console>:32
edgesRDD = ParallelCollectionRDD[1] at parallelize at <console>:33
graph = org.apache.spark.graphx.impl.GraphImpl@785c63fa


org.apache.spark.graphx.impl.GraphImpl@785c63fa

In [3]:
//调用pregel
val finalGraph=Pregel(graph=graph,initialMsg=0,maxIterations=10,activeDirection=EdgeDirection.Out)(
    vprog=(_,vd,aggMsg)=>math.max(vd,aggMsg),
    sendMsg=(edge)=>if(edge.srcAttr+1>edge.dstAttr) Iterator((edge.dstId,edge.srcAttr+1)) else Iterator.empty,
    mergeMsg=(a,b)=>math.max(a,b)
)

finalGraph = org.apache.spark.graphx.impl.GraphImpl@58b17522


org.apache.spark.graphx.impl.GraphImpl@58b17522

In [4]:
finalGraph.vertices.take(10)

Array((4,2), (0,0), (2,1), (1,1), (3,2), (5,0))

In [5]:
可以发现比其上一节的计算要方便很多，说明一下参数含义：   

initialMsg:初始是发送给各节点的消息，它在第一轮调用是对应了vprog函数中的aggMsg;  
maxIterations:最大迭代次数;  
activeDirection:消息传输方向为出度方向，此外还有In,Either,Both这些在后续的算法调用中再介绍;  
vprog:即处理聚合后的消息与原节点属性的函数;  
sendMsg:与aggerateMessages中类似，不过需要在外面套一层Iterator，并指定发送到dstId;  
mergeMsg:与aggrateMessages中一样   

注意，迭代收敛条件似乎是由maxIterations控制的，其实不止，迭代收敛条件还受到sendMsg函数影响，如果某次调用所有边的消息都为Iterator.empty,那么迭代也会终止

Compile Error: <console>:1: error: illegal character '\uff0c'
可以发现比其上一节的计算要方便很多，说明一下参数含义：
                 ^
<console>:1: error: illegal character '\uff1a'
可以发现比其上一节的计算要方便很多，说明一下参数含义：
                          ^
<console>:3: error: illegal character '\uff0c'
initialMsg:初始是发送给各节点的消息，它在第一轮调用是对应了vprog函数中的aggMsg;
                       ^
<console>:5: error: illegal character '\uff0c'
activeDirection:消息传输方向为出度方向，此外还有In,Either,Both这些在后续的算法调用中再介绍;
                           ^
<console>:7: error: illegal character '\uff0c'
sendMsg:与aggerateMessages中类似，不过需要在外面套一层Iterator，并指定发送到dstId;
                            ^
<console>:7: error: illegal character '\uff0c'
sendMsg:与aggerateMessages中类似，不过需要在外面套一层Iterator，并指定发送到dstId;
                                               ^
<console>:10: error: illegal character '\uff0c'
这里，迭代收敛条件似乎是由maxIterations控制的，其实不然，迭代收敛条件还受到sendMsg函数影响，如果某次调用所有边的消息都为Iterator.empty,那么迭代也会终止
  ^
<console>:10: error: illegal character '\uff0c'
这里，迭代收敛条件似乎是由maxIterations控制的，其实不然，迭代